In [ ]:
#Q1
def calculate_smm(pre_payments, beginning_outstanding_principal):
    """
    Single Monthly Mortality (SMM).

    pre_payments: The total prepayments for the month.
    beginning_outstanding_principal: The outstanding principal at the beginning of the month.
    return: The SMM for the month.
    """
    smm = pre_payments / beginning_outstanding_principal
    return smm

def calculate_cpr(smm):
    """
    Conditional Prepayment Rate (CPR) given the SMM.

    smm: The Single Monthly Mortality rate.
    return: The CPR for the month.
    """
    cpr = 1 - (1 - smm)**12
    return cpr

# Inputs
pre_payments = 1.2e6  # $1.2mm
beginning_outstanding_principal = 35e6  # $35mm

# Calculations
smm = calculate_smm(pre_payments, beginning_outstanding_principal)
cpr = calculate_cpr(smm)

print(f"SMM: {smm:.2%}")
print(f"CPR: {cpr:.2%}")


SMM: 3.43%
CPR: 34.21%


In [ ]:
#Q3
# Inputs
original_mbs_amount = 100e6  # $100 million
original_mbs_price = 102  # 102-00 points
tranche_a_amount = 50e6  # $50 million
tranche_a_price = 100  # 100-00 points

# Functions
# Total cost of purchasing the original MBS
total_mbs_cost = original_mbs_amount * (original_mbs_price / 100)

# Revenue from selling Tranche A
tranche_a_revenue = tranche_a_amount * (tranche_a_price / 100)

# To break even, Tranche B revenue + Tranche A revenue = Total MBS cost
# Therefore, we need to calculate the required Tranche B revenue for breakeven
tranche_b_revenue_required = total_mbs_cost - tranche_a_revenue

# Assuming Tranche B also amounts to $50 million, calculate its breakeven price
tranche_b_amount = original_mbs_amount - tranche_a_amount  # This assumes the rest goes into Tranche B
tranche_b_breakeven_price = (tranche_b_revenue_required / tranche_b_amount) * 100  # Convert back to price per 100 nominal

print(f"A) Tranche A revenue: {tranche_a_revenue:.2f}")
print(f"B) Breakeven Price for Tranche B: {tranche_b_breakeven_price:.2f}-00")


A) Tranche A revenue: 50000000.00
B) Breakeven Price for Tranche B: 104.00-00


In [2]:
#Q6
import numpy as np

# Function to calculate monthly payment
def monthly_mortgage_payment(principal, annual_rate, months):
    monthly_rate = annual_rate / 12
    payment = principal * (monthly_rate / (1 - np.power((1 + monthly_rate), -months)))
    return payment

# Function for average life
def average_life(principal, rate, n, cpr=0, years_after=0):
    monthly_payment = monthly_mortgage_payment(principal, rate, n)
    monthly_cpr = 1 - ((1 - cpr) ** (1/12))
    total_months = n
    weighted_life = 0
    remaining_balance = principal

    for month in range(1, total_months + 1):
        # Scheduled principal payment
        interest = remaining_balance * rate / 12
        principal_payment = monthly_payment - interest

        # Prepayment
        prepayment = (remaining_balance - principal_payment) * monthly_cpr

        if month > years_after * 12:
            # Calculate weighted life for months after specified years
            weighted_life += (principal_payment + prepayment) * month / principal

        # Update remaining balance
        remaining_balance -= (principal_payment + prepayment)

        if remaining_balance <= 0:
            break

    return weighted_life / 12  # Convert to years

# a. Assuming no prepayments for life (0 CPR)
average_life_0_cpr = average_life(100_000_000, 0.055, 360, 0)

# b. After 10 years with 0 CPR
average_life_0_cpr_10_years = average_life(100_000_000, 0.055, 360, 0, 10)

# c. Assuming a constant 6 CPR
average_life_6_cpr = average_life(100_000_000, 0.055, 360, 0.06)

# d. After 10 years with constant 6 CPR
average_life_6_cpr_10_years = average_life(100_000_000, 0.055, 360, 0.06, 10)

average_life_0_cpr, average_life_0_cpr_10_years, average_life_6_cpr, average_life_6_cpr_10_years

def years_and_months(years):
    full_years = int(years)  # Get the full years as an integer
    remaining_months = int((years - full_years) * 12)  # Calculate the remaining months
    return full_years, remaining_months

# Convert average life to years and months
average_life_0_cpr_years, average_life_0_cpr_months = years_and_months(average_life_0_cpr)
average_life_0_cpr_10_years_years, average_life_0_cpr_10_years_months = years_and_months(average_life_0_cpr_10_years)
average_life_6_cpr_years, average_life_6_cpr_months = years_and_months(average_life_6_cpr)
average_life_6_cpr_10_years_years, average_life_6_cpr_10_years_months = years_and_months(average_life_6_cpr_10_years)

print(f"A) Assuming no Prepayments for life: {average_life_0_cpr_years} years and {average_life_0_cpr_months} months")
print(f"B) After 10 years with 0 CPR: {average_life_0_cpr_10_years_years} years and {average_life_0_cpr_10_years_months} months")
print(f"C) Assuming a constant 6 CPR: {average_life_6_cpr_years} years and {average_life_6_cpr_months} months")
print(f"D) After 10 years with constant 6 CPR: {average_life_6_cpr_10_years_years} years and {average_life_6_cpr_10_years_months} months")


A) Assuming no Prepayments for life: 18 years and 11 months
B) After 10 years with 0 CPR: 18 years and 0 months
C) Assuming a constant 6 CPR: 6 years and 11 months
D) After 10 years with constant 6 CPR: 3 years and 4 months


In [3]:
# Corrected function to match the results in the picture
def calculate_average_life(pool_balance, wac, monthly_coupon_rate, total_months, cpr=0, after_years=None):
    remaining_balance = pool_balance
    # Monthly interest rate from WAC
    monthly_interest_rate = wac / 12
    # Monthly prepayment rate from annual CPR
    monthly_cpr = 1 - ((1 - cpr) ** (1/12)) if cpr > 0 else 0

    # Array to hold the principal payments
    principal_payments = np.zeros(total_months)
    # Array to hold the time indices
    time_indices = np.arange(1, total_months + 1)

    # Calculate the scheduled payment (assuming no prepayment)
    scheduled_payment = monthly_mortgage_payment(pool_balance, wac, total_months)

    for month in range(total_months):
        # Interest for the month
        interest_payment = remaining_balance * monthly_interest_rate
        # Scheduled principal payment
        scheduled_principal = scheduled_payment - interest_payment
        # Prepayment amount
        prepayment = (remaining_balance - scheduled_principal) * monthly_cpr
        # Actual principal paid
        principal_paid = scheduled_principal + prepayment
        principal_payments[month] = principal_paid
        # Update the remaining balance
        remaining_balance -= principal_paid
        # If the remaining balance goes negative, set it to zero
        if remaining_balance <= 0:
            principal_payments[month] += remaining_balance # Adjust the last payment
            principal_payments = principal_payments[:month+1] # Truncate the array to the last payment
            time_indices = time_indices[:month+1]
            break

    # Calculate the weighted average life of the MBS
    if after_years is not None:
        # If we need the average life after a certain period (years_after), only consider the time and payments after that period
        start_index = after_years * 12
        principal_payments = principal_payments[start_index:]
        time_indices = time_indices[start_index:] - start_index
    weighted_average_life_months = np.sum(principal_payments * time_indices) / np.sum(principal_payments)

    # Convert the average life to years and months
    average_life_years, average_life_months = years_and_months(weighted_average_life_months)

    return average_life_years, average_life_months

# Constants from the problem
pool_balance = 100_000_000  # $100 million
wac = 0.06  # Weighted average coupon rate
monthly_coupon_rate = 0.055 / 12  # Monthly coupon payment rate
total_months = 360  # Total number of months for a 30-year mortgage

# Recalculating average life for each scenario
# a) Assuming no prepayments for life (0 CPR)
average_life_a_years, average_life_a_months = calculate_average_life(pool_balance, wac, monthly_coupon_rate, total_months, 0)

# b) Assuming no prepayments for life (0 CPR), average life after 10 years
average_life_b_years, average_life_b_months = calculate_average_life(pool_balance, wac, monthly_coupon_rate, total_months, 0, 10)

# c) Assuming a constant 6 CPR for life
average_life_c_years, average_life_c_months = calculate_average_life(pool_balance, wac, monthly_coupon_rate, total_months, 0.06)

# d) Assuming a constant 6 CPR, average life after 10 years
average_life_d_years, average_life_d_months = calculate_average_life(pool_balance, wac, monthly_coupon_rate, total_months, 0.06, 10)

# Display the results as per the requested format
average_life_results = {
    'a': (average_life_a_years, average_life_a_months),
    'b': (average_life_b_years, average_life_b_months),
    'c': (average_life_c_years, average_life_c_months),
    'd': (average_life_d_years, average_life_d_months)
}
average_life_results


{'a': (231, 8), 'b': (143, 10), 'c': (82, 9), 'd': (23, 1)}